In [1]:
import numpy as np
from PIL import Image
import time
import matplotlib.pyplot as plt

In [2]:
# just in case we normilize our value
def fix_color(color):
    if 0 < round(color) < 255:
        return round(color)
    elif round(color) >= 255:
        return 255
    else:
        return 0

# it's easier to use flatten pixels because the author uses them in his formulas
def get_flatten_pixels_snapshot(i, j, data):
    return data[i - 2 : i + 3, 
                j - 2 : j + 3, :].reshape(25, -1)

In [3]:
def _compute_green_color_using_base_color(base_color, g):
    delta_N = np.abs(base_color[12] - base_color[2])*2 + np.abs(g[7] - g[17])
    delta_E = np.abs(base_color[12] - base_color[14])*2 + np.abs(g[11] - g[13])
    delta_W = np.abs(base_color[12] - base_color[10])*2 + np.abs(g[11] - g[13])
    delta_S = np.abs(base_color[12] - base_color[22])*2 + np.abs(g[7] - g[17])
    
    smallest_delta = min([delta_N, delta_E, delta_W, delta_S])

    if delta_N == smallest_delta:
        return (g[7] * 3 + g[17] + base_color[12] - base_color[2]) / 4
    elif delta_E == smallest_delta:
        return (g[13] * 3 + g[11] + base_color[12] - base_color[14]) / 4
    elif delta_W == smallest_delta:
        return (g[11] * 3 + g[13] + base_color[12] - base_color[10]) / 4
    else:
        return (g[17] * 3 + g[7] + base_color[12] - base_color[22]) / 4

def compute_all_green_pixels(image):
    # if i is odd that would mean that we are computing row with red and green pixels
    # if i is even that would mean that we are computing row with blue and green pixels
    for i in range(2, image.shape[0] - 2):
        if i % 2 == 0:
            j_start = 2
        else:
            j_start = 3
        for j in range(j_start, image.shape[1] - 2, 2):
            flatten_snapshot = get_flatten_pixels_snapshot(i, j, image)
            
            r = flatten_snapshot[:, 0]
            g = flatten_snapshot[:, 1]
            b = flatten_snapshot[:, 2]
            image[i, j, 1] = fix_color(_compute_green_color_using_base_color(r + b, g))

In [4]:
def hue_transit(l1, l2, l3, v1, v3):
    if l1 < l2 < l3 or l1 > l2 > l3:
        return v1 + (v3 - v1) * (l2 - l1) / (l3 - l1)
    else:
        return (v1 + v3) / 2 + (l2 - (l1 + l3) / 2) / 2

def normalize(x):
    return max(min(round(x), 255), 0)

def stage2_cell_old(col1, col2, g):
    return normalize(hue_transit(g[11], g[12], g[13], col1[11], col1[13])), \
           normalize(hue_transit(g[7], g[12], g[17], col2[7], col2[17]))


def stage2_cell(col1, col2, g):
    return normalize(hue_transit(g[3], g[4], g[5], col1[3], col1[5])), \
           normalize(hue_transit(g[1], g[4], g[7], col2[1], col2[7]))

def get_pattern(data, i, j, k):
    return data[i - (k // 2):i + (k // 2 + 1), j - (k // 2):j + (k // 2 + 1)].flatten()

def stage2(image):
    r = image[:, :, 0]
    g = image[:, :, 1]
    b = image[:, :, 2]
    # Left and Right - Red
    for i in range(2, r.shape[0] - 2, 2):
        for j in range(3, r.shape[1] - 2, 2):
            r_pattern = get_pattern(r, i, j, k=5)
            g_pattern = get_pattern(g, i, j, k=5)
            b_pattern = get_pattern(b, i, j, k=5)
            r[i, j], b[i, j] = stage2_cell(r_pattern, b_pattern, g_pattern)

    # Left and Right - Blue
    for i in range(3, r.shape[0] - 2, 2):
        for j in range(2, r.shape[1] - 2, 2):
            r_pattern = get_pattern(r, i, j, k=5)
            g_pattern = get_pattern(g, i, j, k=5)
            b_pattern = get_pattern(b, i, j, k=5)
            b[i, j], r[i, j] = stage2_cell(b_pattern, r_pattern, g_pattern)
            
    return np.dstack((r[2:-2, 2:-2], g[2:-2, 2:-2], b[2:-2, 2:-2]))


In [5]:
def hue_transit(l1, l2, l3, v1, v3):
    if l1 < l2 < l3 or l1 > l2 > l3:
        return v1 + (v3 - v1) * (l2 - l1) / (l3 - l1)
    else:
        return (v1 + v3) / 2 + (l2 - (l1 + l3) / 2) / 2
    
def compute_red_and_blue_colors_for_green_pixels(image):
    for i in range(2, image.shape[0] - 2):
        if i % 2 != 0:
            j_start = 2
        else:
            j_start = 3
        for j in range(j_start, image.shape[1] - 2, 2):
            flatten_snapshot = get_flatten_pixels_snapshot(i, j, image)
            
            r = flatten_snapshot[:, 0]
            g = flatten_snapshot[:, 1]
            b = flatten_snapshot[:, 2]
            if i % 2 != 0:
                image[i, j, 0] = fix_color(hue_transit(g[7], g[12], g[17], r[7], r[17]))
                image[i, j, 2] = fix_color(hue_transit(g[11], g[12], g[13], b[11], b[13]))
            else:
                image[i, j, 0] = fix_color(hue_transit(g[11], g[12], g[13], r[11], r[13]))
                image[i, j, 2] = fix_color(hue_transit(g[7], g[12], g[17], b[7], b[17]))

In [6]:
def _compute_red_or_blue_color_using_base_and_adjacent_colors(adjacent_color, base_color, g):
        delta_NE = np.abs(adjacent_color[8] - adjacent_color[16])  + np.abs(base_color[4] - base_color[12]) + np.abs(base_color[12] - base_color[20]) + np.abs(g[8] - g[12]) + np.abs(g[12] - g[16])
        
        delta_NW = np.abs(adjacent_color[6] - adjacent_color[18])  + np.abs(base_color[0] - base_color[12]) + np.abs(base_color[12] - base_color[24])  + np.abs(g[6] - g[12]) + np.abs(g[12] - g[18])

        if delta_NE < delta_NW:
            return hue_transit(g[8], g[12], g[16], adjacent_color[8], adjacent_color[16])
        else:
            return hue_transit(g[6], g[12], g[18], adjacent_color[6], adjacent_color[18])


def compute_red_for_blue_pixel_and_blue_for_red_pixel(image):
    for i in range(2, image.shape[0] - 2):
        if i % 2 == 0:
            j_start = 2
        else:
            j_start = 3
        for j in range(j_start, image.shape[1] - 2, 2):
            flatten_snapshot = get_flatten_pixels_snapshot(i, j, image)
            
            r = flatten_snapshot[:, 0]
            g = flatten_snapshot[:, 1]
            b = flatten_snapshot[:, 2]
            if j_start == 2:
                # compute blue color for red pixel
                image[i, j, 2] = fix_color(_compute_red_or_blue_color_using_base_and_adjacent_colors(b, r, g))
            else:
                # compute red color blue red pixel
                image[i, j, 0] = fix_color(_compute_red_or_blue_color_using_base_and_adjacent_colors(r, b, g))

In [7]:
def ppg_algorithm(input_img):
    # make boundaries for image
    data = np.zeros((input_img.shape[0] + 4, input_img.shape[1] + 4, 3))
    data[2:-2, 2:-2] = input_img
    compute_all_green_pixels(data)
    compute_red_and_blue_colors_for_green_pixels(data)    
    compute_red_for_blue_pixel_and_blue_for_red_pixel(data)
    return data[2:-2, 2:-2, :]

In [8]:
from cv2 import cvtColor, COLOR_RGB2GRAY

def mse(pic1, pic2):
    y1 = cvtColor(pic1.astype('uint8'), COLOR_RGB2GRAY)
    y2 = cvtColor(pic2.astype('uint8'), COLOR_RGB2GRAY)
    return np.mean((y1-y2) ** 2)


def psnr(pic1, pic2):
    return 10*np.log10(255**2 / mse(pic1, pic2))

In [13]:
input_img_name = 'RGB_CFA.bmp'
target_img = 'Original.bmp'

img = np.asarray(Image.open(input_img_name), dtype=np.float32)
target_img = np.asarray(Image.open(target_img), dtype=np.float32)
start = time.time()
result_img = ppg_algorithm(img)
execution_time = time.time() - start

print("Время выполнения: " + str(execution_time))
print("Скорость выполнения: " + str(execution_time / (result_img.shape[0] * result_img.shape[1]) * 1e6) + str("сек/мегапиксель"))
print("MSE: " + str(mse(result_img, target_img)))
print("PSNR: " + str(psnr(result_img, target_img)))

im = Image.fromarray(result_img.astype(np.uint8))
im.save('Result.bmp')

Время выполнения: 265.4816164970398
Скорость выполнения: 30.667237832642986сек/мегапиксель
MSE: 35.25471049046951
PSNR: 32.658632082399805
